In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import bs4 as bs4
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
import urllib

In [56]:
driver = webdriver.Edge()
driver.get('https://spsepchtr.mx.toyota.co.jp/kN0k/V2/servlet/jp.co.toyota.pv2y580.servlet.CCV2Y580')
user_input = driver.find_element(By.XPATH,'/html/body/div/div[2]/div[2]/div/div/form/div[2]/div[1]/table/tbody/tr[1]/td[2]/input')
user_input.send_keys('12062B5@97221')
pwd_input = driver.find_element(By.XPATH, '/html/body/div/div[2]/div[2]/div/div/form/div[2]/div[1]/table/tbody/tr[2]/td[2]/input')
pwd_input.send_keys('Gpauto555#')
pwd_input.send_keys(Keys.ENTER)

In [57]:
confirm_login_button = driver.find_element(By.XPATH,'/html/body/div/div[2]/div/div[2]/input[1]')
confirm_login_button.click()

In [58]:
# Wait for the new window/tab and then switch to it
window_handles = driver.window_handles
driver.switch_to.window(window_handles[-1])

In [59]:
#Get into the menu to search parts.
iframe = driver.find_element(By.XPATH,'/html/body/div/iframe')
driver.switch_to.frame(iframe)
partSearch_button = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[3]/div[1]/fieldset/button[2]')
partSearch_button.click()

In [60]:
#Get into iframe that contain input box
driver.switch_to.default_content()
iframe = driver.find_element(By.XPATH,'/html/body/div[2]/iframe')
driver.switch_to.frame(iframe)

<font color='yellow' size = 5>Inventory</font>

In [54]:
GPInventory = pd.read_excel(r'GP.xlsx')
GPInventory = GPInventory.loc[GPInventory['ItemGroup'].apply(lambda x : str(x).find('TOYOTA'))>=0][['ItemCode']]

In [55]:
GPInventory = GPInventory.drop_duplicates()

In [56]:
GPInventory.loc[:,'ItemCode'] = GPInventory['ItemCode'].apply(lambda x : x[4:] )

In [57]:
GPInventory = GPInventory.loc[GPInventory['ItemCode'].apply(lambda x : len(x)).isin([10,12])]

In [58]:
GPInventory.columns = ['Code']

In [59]:
GPInventory = GPInventory.reset_index()

In [60]:
GPInventory = GPInventory.drop(['index'], axis=1)

<font color='lightgreen' size=5> Inventory Full Version</font>

In [14]:
GPInventory = pd.read_csv(r'/Users/krichnorakhun/Documents/GP/inventory/TOY-CompleteList.csv')

In [16]:
GPInventory = GPInventory[['PartNO']]

In [17]:
GPInventory.columns = ['Code']
GPInventory = GPInventory.reset_index()
GPInventory = GPInventory.drop(['index'], axis=1)

<font color='yellow' size=5> Scrapping Part </font>

In [19]:
def clickInput10(x):
    #Get input box
    part_1 = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[1]/div[2]/input[1]')
    part_2 = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[1]/div[2]/input[2]')
    part_3 = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[1]/div[2]/input[3]')
    part_1.send_keys(x[:6])
    part_2.send_keys(x[6:11])
    #part_3.send_keys(x[11:])
    part_3.send_keys(Keys.ENTER)
    Run_Botton = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[3]/div[2]/button[10]')
    Run_Botton.click()
    return

In [20]:
def clickInput12(x):
    #Get input box
    part_1 = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[1]/div[2]/input[1]')
    part_2 = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[1]/div[2]/input[2]')
    part_3 = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[1]/div[1]/div[2]/input[3]')
    part_1.send_keys(x[:6])
    part_2.send_keys(x[6:11])
    part_3.send_keys(x[11:])
    part_3.send_keys(Keys.ENTER)
    Run_Botton = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[3]/div[2]/button[10]')
    Run_Botton.click()
    return

In [21]:
def getResult_1(x,output):
    tempData = pd.DataFrame()
    soup = bs4.BeautifulSoup(x)
    tableInSoup = soup.find_all('table',{'class':'excelLikeList'})
    tempData.loc[0, 'PartID'] = tableInSoup[0].find_all('div', { 'class':'side'})[0].text
    tempData.loc[0, 'Part Description'] = tableInSoup[0].find_all('div', { 'class':'side'})[1].text
    output = pd.concat([output,tempData], ignore_index=True)
    return(output,tableInSoup[0].find_all('div', { 'class':'side'})[0].text)

In [22]:
def clickNewInput():
    newFind = driver.find_element(By.XPATH,'/html/body/form/div[1]/div[3]/div[2]/button[1]')
    newFind.click()


In [23]:
def getResult_2(data,output):
    soup = bs4.BeautifulSoup(data)
    tableInSoup = soup.find_all('table',{'class':'excelLikeList'})
    tempData = pd.DataFrame()
    for x in tableInSoup[1].find_all('tr'):
        tempData.loc[0,'PartID'] = x.find_all('td')[1].text
        tempData.loc[0,'PNC'] = x.find_all('td')[2].text
        tempData.loc[0,'Count'] = x.find_all('td')[3].text
        tempData.loc[0,'Catalogue'] = x.find_all('td')[4].text
        tempData.loc[0,'Model'] = x.find_all('td')[5].text
        tempData.loc[0,'Fr-To'] = x.find_all('td')[6].text
        tempData.loc[0,'Model(Details)'] = x.find_all('td')[7].text
        output = pd.concat([output, tempData], ignore_index=True)
    return(output)
    #print(x.find_all('td')[1].text)

In [45]:
def getResult_2pics(data,output,imgName,thirdTable,PartID):
    soup = bs4.BeautifulSoup(data)
    tableInSoup = soup.find_all('table',{'class':'excelLikeList'})
    
    tempData_List = []
    for x in tableInSoup[1].find_all('tr'):
        # tempData.loc[0,'PartID'] = x.find_all('td')[1].text
        # tempData.loc[0,'PNC'] = x.find_all('td')[2].text
        # tempData.loc[0,'Count'] = x.find_all('td')[3].text
        # tempData.loc[0,'Catalogue'] = x.find_all('td')[4].text
        # tempData.loc[0,'Model'] = x.find_all('td')[5].text
        # tempData.loc[0,'Fr-To'] = x.find_all('td')[6].text
        # tempData.loc[0,'Model(Details)'] = x.find_all('td')[7].text
        
        tempData_List.append({
        'PartID': PartID, #x.find_all('td')[1].text,
        'PNC': x.find_all('td')[2].text,
        'Count': x.find_all('td')[3].text,
        'Catalogue': x.find_all('td')[4].text,
        'Model': x.find_all('td')[5].text,
        'Fr-To' : x.find_all('td')[6].text,
        'Model(Details)' : x.find_all('td')[7].text
        })

        #output = pd.concat([output, tempData], ignore_index=True)
        
        #for add to Third Table
        #PartID = x.find_all('td')[1].text
        PNC = x.find_all('td')[2].text

    tempData = pd.DataFrame(tempData_List)
    output = pd.concat([output, tempData], ignore_index=True)        
    
    #Click First Row
    firtRowToClickID = soup.find_all('table',{'id':'F10_tblListInnerSearch'})[0].find_all('tr')[0].get('id')
    driver.find_element(By.ID,firtRowToClickID).click()

    #Click button to open new window to show picture
    driver.find_element(By.XPATH,'/html/body/form/div[1]/div[3]/div[2]/button[8]').click()
    
    # Wait for a new window/tab to open
    wait = WebDriverWait(driver, 60)
    current_windows = driver.window_handles
    wait.until(lambda driver: len(driver.window_handles) > len(current_windows))

    # Switch to the new window/tab
    new_window_handles = driver.window_handles
    driver.switch_to.window(new_window_handles[-1])
        
    #Get into iframe where picture located.
    iframe = driver.find_element(By.XPATH,'/html/body/div/iframe')
    driver.switch_to.frame(iframe)

    # Wait for the image to be present
    wait = WebDriverWait(driver, 30)  # Adjust the timeout as needed
    img_element = wait.until(EC.presence_of_element_located((By.ID, 'figImgFig')))

    # Now retrieve the image URL
    img_url = img_element.get_attribute('src')
    urllib.request.urlretrieve(img_url,'images/'+imgName+'.png')

    #Get PNC list on image and its location
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    
    tempdata2_List = []
    width = float(soup.find_all('img',{'id':'figImgFig'})[0].get('style').split(';')[1].split(':')[1].replace('px','').strip())
    height = float(soup.find_all('img',{'id':'figImgFig'})[0].get('style').split(';')[2].split(':')[1].replace('px','').strip())
    figureName = soup.find_all('div',{'id':'D13_tblThumbnailtmnFigname_0_0'})[0].text
    for pnc in soup.find_all('div',{'id':'figInner'})[0].find_all('div'):
        top = float(pnc.get('style').split(';')[0].split(':')[1].replace('px', ''))
        left = float(pnc.get('style').split(';')[1].split(':')[1].replace('px', ''))
        fontsize = float(pnc.get('style').split(';')[2].split(':')[1].replace('px',''))
        lineheight = float(pnc.get('style').split(';')[3].split(':')[1].replace('px',''))
        transform = pnc.get('style').split(';')[4].split(':')[1].strip()        
        text = pnc.text
         # Append to the list as a dictionary
        tempdata2_List.append({
        'PartID': PartID,
        'PNC': PNC,
        'Top': top,
        'Left': left,
        'Fontsize':fontsize,
        'LineHeight':lineheight,
        'Transform':transform,
        'Width':width,
        'Height':height,
        'Text': text,
        'figureName':figureName
        })
    tempData2 = pd.DataFrame(tempdata2_List)
    thirdTable = pd.concat([thirdTable, tempData2], ignore_index=True)

    #Close window and switch back to previous window.
    driver.close()
    # Wait for the new window/tab and then switch to it
    window_handles = driver.window_handles
    driver.switch_to.window(window_handles[-1])

    iframe = driver.find_element(By.XPATH,'/html/body/div[2]/iframe')
    driver.switch_to.frame(iframe)
    
    return output,thirdTable
    #print(x.find_all('td')[1].text)

<font color='yellow' size=5> simulate save picture </font>

In [25]:
clickInput12('0411130544')

In [26]:
WebDriverWait(driver,60)
data = driver.page_source
soup = bs4.BeautifulSoup(data)

In [27]:
firtRowToClickID = soup.find_all('table',{'id':'F10_tblListInnerSearch'})[0].find_all('tr')[0].get('id')

In [28]:
driver.find_element(By.ID,firtRowToClickID).click()

In [29]:
#Click to open new window to show picture
driver.find_element(By.XPATH,'/html/body/form/div[1]/div[3]/div[2]/button[8]').click()
# Wait for a new window/tab to open
wait = WebDriverWait(driver, 60)
current_windows = driver.window_handles
wait.until(lambda driver: len(driver.window_handles) > len(current_windows))

True

In [32]:
# Switch to the new window/tab
new_window_handles = driver.window_handles
driver.switch_to.window(new_window_handles[-1])

In [33]:
#Get into iframe where picture located.
iframe = driver.find_element(By.XPATH,'/html/body/div/iframe')
driver.switch_to.frame(iframe)

In [34]:
# Wait for the image to be present
wait = WebDriverWait(driver, 30)  # Adjust the timeout as needed
img_element = wait.until(EC.presence_of_element_located((By.ID, 'figImgFig')))

# Now retrieve the image URL
img_url = img_element.get_attribute('src')
print(img_url)

https://spsepchtr.mx.toyota.co.jp/kN0k/CONTENTS/HBUNKAI/113494B/113494B.PNG


In [35]:
data = driver.page_source
soup = bs4.BeautifulSoup(data)

In [44]:
soup.find_all('div',{'id':'D13_tblThumbnailtmnFigname_0_0'})[0].text

'ชุดปะเก็นโอเวอร์ฮอลเครื่องยนต์'

In [54]:
width = float(soup.find_all('img',{'id':'figImgFig'})[0].get('style').split(';')[1].split(':')[1].replace('px','').strip())

In [60]:
height = float(soup.find_all('img',{'id':'figImgFig'})[0].get('style').split(';')[2].split(':')[1].replace('px','').strip())

492.0

In [31]:
pnc = soup.find_all('div',{'id':'figInner'})[0].find_all('div')[0]

In [51]:
pnc.get('style')#.split(';')[4].split(':')[1].strip()

'top: 243.024px; left: 156.81px; font-size: 8.09647px; line-height: 8.09647px; transform: scale(1.1, 1.25);'

In [412]:
tempData2 = pd.DataFrame()
for pnc in soup.find_all('div',{'id':'figInner'})[0].find_all('div'):
    tempData2['PartID'] = PartID
    tempData2['PNC'] = PNC
    tempData2['Top'] = float(pnc.get('style').split(';')[0].split(':')[1].replace('px',''))
    tempData2['Left'] = float(pnc.get('style').split(';')[1].split(':')[1].replace('px',''))
    
    tempData2['Text'] = pnc.text
    
    

top: 243.024px; left: 156.81px; font-size: 8.09647px; line-height: 8.09647px; transform: scale(1.1, 1.25);
156.81
04112
top: 6.02412px; left: 155.81px; font-size: 8.09647px; line-height: 8.09647px; transform: scale(1.1, 1.25);
155.81
04111


In [395]:
soup.find_all('div',{'id':'figInner'})[0].find_all('div')[0].get('style')

'top: 243.024px; left: 156.81px; font-size: 8.09647px; line-height: 8.09647px; transform: scale(1.1, 1.25);'

In [92]:
driver.close()

In [27]:
# Wait for the new window/tab and then switch to it
window_handles = driver.window_handles
driver.switch_to.window(window_handles[-1])

In [28]:
#Get into iframe that contain input box
iframe = driver.find_element(By.XPATH,'/html/body/div[2]/iframe')
driver.switch_to.frame(iframe)

In [210]:
bs4.BeautifulSoup(driver.page_source).find_all('title')

[<title>Part Number Application to Models</title>]

In [159]:
soup.find_all('title')

[<title>Part Number Application to Models</title>]

<font color='yellow' size=5>RUN</font>

In [61]:
MainTable = pd.DataFrame()
SecondTable = pd.DataFrame()
ThirdTable = pd.DataFrame()

In [62]:
def clearCheckbox():
    driver.find_element(By.ID,'F10_chkSmksKbnJA').click()
    driver.find_element(By.ID,'F10_chkSmksKbnEU').click()
    driver.find_element(By.ID,'F10_chkSmksKbnNA').click()

In [63]:
for index, row in GPInventory.iloc[:10].iterrows():
    
    if len(row['Code'].strip())==10:
        clickInput10(row['Code'].strip())
    else:
        clickInput12(row['Code'].strip())
    
    wait = WebDriverWait(driver, 60) 
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)

    try:
        wait.until(EC.visibility_of_element_located((By.ID, 'eltColF10_divListSelectHinban4')))
        data = driver.page_source
        soup = bs4.BeautifulSoup(data)
        if soup.find_all('div',{'id':'eltColF10_divListSelectHinban4'})[0].text != 'ไม่มีใช้งาน':
            wait.until(EC.visibility_of_element_located((By.ID, 'eltRowF10_divListSearch1')))
       
        #time.sleep(1)
        data = driver.page_source
        MainTable, PartID = getResult_1(data,MainTable)
        if soup.find_all('div',{'id':'eltColF10_divListSelectHinban4'})[0].text != 'ไม่มีใช้งาน':
            SecondTable, ThirdTable = getResult_2pics(data,SecondTable,row['Code'].strip(),ThirdTable,PartID )
        print(index)
        
    except TimeoutException:
        print("Timeout occurred! Performing alternative steps.")

    finally:
        clickNewInput()
        clearCheckbox()
        

0
1
2
3
4
5
6
7
8
9


In [473]:
#Get into iframe that contain input box
driver.switch_to.default_content()
iframe = driver.find_element(By.XPATH,'/html/body/div[2]/iframe')
driver.switch_to.frame(iframe)

In [ ]:
clickNewInput()

In [37]:
MainTable1 = MainTable.drop_duplicates()

In [75]:
MainTable1.to_excel(r'/Users/krichnorakhun/Documents/GP/inventory/EPC1_w_images.xlsx', index=False)

In [41]:
MainTable1['PartID'].apply(lambda x : len(x.replace('-',''))).unique()

array([10, 12], dtype=int64)

In [42]:
SecondTable1 = SecondTable.drop_duplicates()

In [77]:
SecondTable1.to_excel(r'/Users/krichnorakhun/Documents/GP/inventory/EPC2_w_images.xlsx', index=False)

In [43]:
SecondTable1['PartID'].apply(lambda x : len(x.replace('-',''))).unique()

array([10, 12, 24, 20], dtype=int64)

In [33]:
ThirdTable1 = ThirdTable.drop_duplicates()

In [34]:
ThirdTable1.to_csv(r'D:/EPC3_w_images.csv', index=False)

In [64]:
MainTable

,PartID,Part Description
0,00008-54010-02,ไม่มีใช้งาน
1,00008-54010-04,ไม่มีใช้งาน
2,00008-54010-17,ไม่มีใช้งาน
3,04004-07122,ไม่มีใช้งาน
4,04006-0210D,ไม่มีใช้งาน
5,04007-76133,ไม่มีใช้งาน
6,04111-0C105,"ชุดปะเก็น, การซ่อมใหญ่เครื่องยนต์"
7,04111-0C109,"ชุดปะเก็น, การซ่อมใหญ่เครื่องยนต์"
8,04111-0C126,"ชุดปะเก็น, การซ่อมใหญ่เครื่องยนต์"
9,04111-0C136,ไม่มีใช้งาน


In [65]:
SecondTable

,PartID,PNC,Count,Catalogue,Model,Fr-To,Model(Details)
0,04111-0C105,04111(ทดแทน),0101,6713A0,FORTUNER,201204- 201206,2TRFE..TGN61..HTWC..LA
1,04111-0C105,04111(ทดแทน),0101,6713A0,FORTUNER,201204- 201208,2TRFE..TGN61..TH
2,04111-0C105,04111(ทดแทน),0101,672380,HILUX,201204- 201206,2TRFE..TGN26..THDCB..DLX2TRFE..TGN36..HTWC..(G...
3,04111-0C105,04111(ทดแทน),0101,672380,HILUX,201204- 201206,"2TRFE..TGN16,26..DCB..5F..DLX..GCCEGYPT SPEC"
4,04111-0C109,04111,01,6713A0,FORTUNER,201412- 201502,2TRFE..TGN61..TH
5,04111-0C109,04111,01,6713A0,FORTUNER,201412- 201503,"2TRFE..TGN51,61..HTWC..(KZS,LA,MA)"
6,04111-0C109,04111,01,6713A0,FORTUNER,201412- 201503,2TRFE..TGN61..GENSINGAPORE SPEC
7,04111-0C109,04111,01,672380,HILUX,201412- 201502,2TRFE..TGN16..RHD2TRFE..TGN26..THDCB..DLX2TRFE...
8,04111-0C109,04111,01,672380,HILUX,201412- 201502,"2TRFE..TGN16,26..DCB..5F..DLX..GCCEGYPT SPEC"
9,04111-0C109,04111,01,672380,HILUX,201412- 201503,"2TRFE..TGN26,36..HTWC..LA"


In [75]:
SecondTable.loc[SecondTable['PartID']=='27060-0C021']

,PartID,PNC,Count,Catalogue,Model,Fr-To,Model(Details)
21770,27060-0C021,27020,01,611260,INNOVA/KIJANG INNOVA,201003- 201204,"1TRFE,2TRFE..TGN4#..GCC,IDN,MA,PHL,TH,VN12V 80A"
21771,27060-0C021,27020,01,611270,INNOVA/KIJANG INNOVA,201107- 201204,2TRFE..TGN4112V 80A
21772,27060-0C021,27020,01,611270,INNOVA/KIJANG INNOVA,201107- 201212,"1TRFE..TGN40..(GL,STD)..VN1TRFE..TGN40..4FC..D..."
21773,27060-0C021,27020,01,611270,INNOVA/KIJANG INNOVA,201107- 201304,"1TRFE..TGN40..IDN,TH1TRFE..TGN40..5F..DLX..VN1..."
21774,27060-0C021,27020,01,611270,INNOVA/KIJANG INNOVA,201204- 201304,2TRFE..TGN4112V 80A
21775,27060-0C021,27020,01,611270,INNOVA/KIJANG INNOVA,201107- 201603,"1TRFE..TGN40..MA,PHL12V 80A"
21776,27060-0C021,27020,01,611270,INNOVA/KIJANG INNOVA,201304- 201603,2TRFE..TGN4112V 80A
21777,27060-0C021,27020,01,671380,FORTUNER,201003- 201204,"2TRFE..TGN51,6112V 80A"
21778,27060-0C021,27020,01,6713A0,FORTUNER,201107- 201204,"2TRFE..TGN51,61..GCC,PHL12V 80A"
21779,27060-0C021,27020,01,6713A0,FORTUNER,201107- 201304,"2TRFE..TGN51,61..IDN,TH,VN12V 80A"


In [66]:
ThirdTable

,PartID,PNC,Top,Left,Fontsize,LineHeight,Transform,Width,Height,Text,figureName
0,04111-0C105,04111(ทดแทน),13.02460,157.81,8.09856,8.09856,"scale(1.1, 1.25)",339.0,493.0,04111,ชุดปะเก็นโอเวอร์ฮอลเครื่องยนต์
1,04111-0C105,04111(ทดแทน),28.02460,157.81,8.09856,8.09856,"scale(1.1, 1.25)",339.0,493.0,04112,ชุดปะเก็นโอเวอร์ฮอลเครื่องยนต์
2,04111-0C109,04111,13.02460,157.81,8.09856,8.09856,"scale(1.1, 1.25)",339.0,493.0,04111,ชุดปะเก็นโอเวอร์ฮอลเครื่องยนต์
3,04111-0C109,04111,28.02460,157.81,8.09856,8.09856,"scale(1.1, 1.25)",339.0,493.0,04112,ชุดปะเก็นโอเวอร์ฮอลเครื่องยนต์
4,04111-0C126,04111(ทดแทน),243.02500,156.81,8.09856,8.09856,"scale(1.1, 1.25)",339.0,493.0,04112,ชุดปะเก็นโอเวอร์ฮอลเครื่องยนต์
5,04111-0C126,04111(ทดแทน),6.02464,156.81,8.09856,8.09856,"scale(1.1, 1.25)",339.0,493.0,04111,ชุดปะเก็นโอเวอร์ฮอลเครื่องยนต์
